In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
    datasets, ds_info = tfds.load(name='mnist', with_info = True, as_supervised = True)
    mnist_train, mnist_test = datasets['train'], datasets['test']
    
    BUFFER_SIZE = 10000
    
    BATCH_SIZE_PER_REPLICA = 64
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync
    
    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        
        return image, label
    
    train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)
    
    return train_dataset, eval_dataset

def get_model():
    with mirrored_strategy.scope():
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32,3,activation = 'relu', input_shape=(28,28,1)),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation = 'relu'),
            tf.keras.layers.Dense(10)
        ])
        
        model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                      optimizer = tf.keras.optimizers.Adam(),
                      metrics = ['accuracy'])
        return model

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

Epoch 1/2
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

938/938 [==============================] - 3s 4ms/step - loss: 0.2016 - accuracy: 0.9418
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Epoch 2/2
938/938 [==============================] - 3s 3ms/step - loss: 0.0644 - accuracy: 0.9808


In [12]:
keras_model_path = 'tmp/keras_save'
model.save(keras_model_path)

INFO:tensorflow:Assets written to: tmp/keras_save\assets


INFO:tensorflow:Assets written to: tmp/keras_save\assets


In [13]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 3s 3ms/step - loss: 0.0469 - accuracy: 0.0989
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - loss: 0.0318 - accuracy: 0.0989


In [8]:
another_strategy = tf.distribute.OneDeviceStrategy('/cpu:0')
with another_strategy.scope():
    restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
    restored_keras_model_ds.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0468 - accuracy: 0.0989
Epoch 2/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0326 - accuracy: 0.0988


In [14]:
model = get_model()
saved_model_path = 'tmp/tf_save'
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: tmp/tf_save\assets


INFO:tensorflow:Assets written to: tmp/tf_save\assets


In [15]:
DEFAULT_FUNCTION_KEY = 'serving_default'
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

In [16]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
    print(inference_func(batch))

{'dense_3': <tf.Tensor: shape=(64, 10), dtype=float32, numpy=
array([[-0.0829123 , -0.06152943, -0.01587072, -0.1656029 ,  0.05474384,
         0.03666794,  0.00277608,  0.02019307, -0.10257424,  0.13552004],
       [-0.33277196, -0.00605263, -0.02691737, -0.27874362,  0.04871486,
         0.07691699, -0.07253227,  0.26639998, -0.18273337,  0.16780001],
       [-0.26240134, -0.04634908,  0.01078824, -0.23441333,  0.16879407,
         0.04412546,  0.06927565, -0.01572404, -0.2222803 ,  0.07767498],
       [ 0.00459268,  0.07381551, -0.03296474, -0.09470411, -0.12175172,
        -0.01561971, -0.03131202,  0.05366429,  0.0007196 , -0.02906748],
       [-0.18901592, -0.00352631, -0.02599388, -0.28089857, -0.07095713,
         0.0234315 , -0.10426559,  0.11795697, -0.01998066,  0.13857779],
       [-0.09944651, -0.05678781,  0.02039987, -0.09451534,  0.09430286,
        -0.02301737,  0.00447764,  0.02802629, -0.20443335,  0.02968082],
       [-0.20867878,  0.01297967, -0.0015285 , -0.227016

In [17]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
    loaded = tf.saved_model.load(saved_model_path)
    inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]
    
    dist_predict_dataset = another_strategy.experimental_distribute_dataset(predict_dataset)
    
    for batch in dist_predict_dataset:
        another_strategy.run(inference_func, args=(batch,))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [22]:
import tensorflow_hub as hub

def build_model(loaded):
    x = tf.keras.layers.Input(shape=(28,28,1), name='input_x')
    
    keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
    model = tf.keras.Model(x, keras_layer)
    return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
    loaded = tf.saved_model.load(saved_model_path)
    model = build_model(loaded)
    
    model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  optimizer = tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    model.fit(train_dataset, epochs=2)
    

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Epoch 1/2
938/938 [==============================] - 3s 3ms/step - loss: 0.2107 - accuracy: 0.9386
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - loss: 0.0681 - accuracy: 0.9797


In [23]:
model = get_model()

model.save(keras_model_path)

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
    loaded = tf.saved_model.load(keras_model_path)

INFO:tensorflow:Assets written to: tmp/keras_save\assets


INFO:tensorflow:Assets written to: tmp/keras_save\assets


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [27]:
class SubclassedModel(tf.keras.Model):
    output_name = 'output_layer'
    
    def __init__(self):
        super(SubclassedModel, self).__init__()
        self._dense_layer = tf.keras.layers.Dense(5, dtype=tf.dtypes.float32, name=self.output_name)
        
    def call(self, inputs):
        return self._dense_layer(inputs)
    
my_model = SubclassedModel()
#my_model.save(keras_model_path)
tf.saved_model.save(my_model, saved_model_path)

INFO:tensorflow:Assets written to: tmp/tf_save\assets


INFO:tensorflow:Assets written to: tmp/tf_save\assets
